In [1]:
import pandas as pd
import pyarrow.parquet as pq
from dask import dataframe as dd
import time
import dask.dataframe as dd
import requests
import os
import dask.array as da
import os
import requests
import shutil
import dask.dataframe as dd
import time


In [2]:
dask_photo_df = dd.read_csv('../Downloading_images/photos.csv/photos.csv')

In [3]:
# Split the single column into multiple columns using the separator '\t'
dask_photos_df_split = dask_photo_df[dask_photo_df.columns[0]].str.split('\t', n=8, expand=True)

# Define column names
column_names = ['photo_uuid', 'photo_id', 'observation_uuid', 'observer_id', 'extension', 'license', 'width', 'height', 'position']

# Rename columns
dask_photos_df_split.columns = column_names

# Define meta for the taxon_id column
meta = ('photo_id', 'float64')

# Convert taxon_id to numeric using apply with meta
dask_photos_df_split['photo_id'] = dask_photos_df_split['photo_id'].apply(pd.to_numeric, errors='coerce', meta=meta)

# Drop rows with NaN in taxon_id column
dask_photos_df_split = dask_photos_df_split.dropna(subset=['photo_id'])

In [4]:
dask_photos_df_split.head(2) 

,photo_uuid,photo_id,observation_uuid,observer_id,extension,license,width,height,position
0,8d6b2534-d30a-47a8-bc1c-986a21817997,21213,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,jpg,CC-BY-NC,584,389,0
1,6e8112fd-f703-4052-94da-b7cfc03ff3d4,21216,7ae155fc-f49e-4e4f-91c5-51e31e805478,516,jpg,CC-BY-NC,584,389,1


In [5]:
image_dir = '../Downloading_images/data/'

In [6]:

# Function to check if an image exists in any subfolder
def image_exists_in_subfolders(photo_id):
    for root, dirs, files in os.walk(image_dir):
        if f'{photo_id}.jpg' in files:
            return True
    return False

In [7]:
# Specify the photo_id you want to search for
desired_photo_id = 94354  

In [9]:
# Check if the desired photo_id exists in any subfolder
if image_exists_in_subfolders(desired_photo_id):
    # If the image exists, find the corresponding row in the DataFrame
    desired_rows = dask_photos_df_split[dask_photos_df_split['photo_id'] == desired_photo_id]
    if len(desired_rows.index) > 0:
        print(desired_rows.compute())  # Compute to get the actual data from Dask DataFrame
    else:
        print("No row found for the specified photo_id.")
else:
    print("Image not found for the specified photo_id.")


: 